---

# OFDM Python Lab

The goal of this lab is to understand and implement the OFDM physical layer for the 802.11a standard, more commonly known as Wi-Fi.

You are provided with an OFDM signal coming from a multipath, noisy channel that is received (<code>homework_x.npy</code>) and you need to implement the receiver part of the OFDM modem. We will guide you through the different components that need to be implemented in this notebook. Your task in this lab is to fill in the code boxes inside the regions marked as <font color='red'>#YOUR CODE STARTS HERE#</font> and <font color='red'>#YOUR CODE ENDS HERE#</font>. 

Each code box is given a unique number like "Box 1" in order to be used as reference in the instructions. 

For this, you will need to code the following RX signal chain.

* Cyclic Prefix Removal  
* Fast Fourrier Transform (FFT)
* Channel Estimation
* Channel Equalization
* Data Decoding


---


## Data Extraction
First of all we need to import the libraries we will need in this lab. Note that you are free to add any additional libraries that you might need based on your own implementation. 
The "helper_functions" library is a library of some functions that we have written ourselves.

### Box 1:

In [184]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import helper_functions as hf

# Import the additional libraries you might use (if any)
# YOUR CODE STARTS HERE #

# YOUR CODE ENDS HERE #

Let's start by opening our homework assignment. You have to request the file from us by sending an email. Make sure you change the "x" with the number assigned to you.

### Box 2:

In [185]:
# Change the signal file name to reflect your assignment
# YOUR CODE STARTS HERE #
filename = "homeworkx.npy"
# YOUR CODE ENDS HERE #

try:
    rx_signal = np.load(filename)
except:
    print("Please change the filename variable to your homework assignment which is formatted as \"homework00.npy\"")


Please change the filename variable to your homework assignment which is formatted as "homework00.npy"


Let's start by defining the basic parameters of our lab, that you will need to use all along the signal chain, which are also known to the transmitter. Note that in this homework, the cyclic prefix is added before the OFDM symbols.

### Box 3:

In [186]:
fs = 20e6 # Sampling rate 
N = 64 # No. of OFDM sub-carriers 

n_data = 52 # No. of data sub-carriers
cp_len = 16 # No. of cyclic prefix samples

n_ofdm = 1000 # No. of OFDM symbols
nsym = n_ofdm * n_data # No. of data symbols

The preamble bits are used for channel estimation and are known to both the transmitter and the receiver. They are pre-defined and shared between the transmitter and the receiver. Here we actually do not define the bits themselves, but we define the modulated result of the bits (using BPSK). So there is no need to demodulate the received preamble symbols to estimate the channel. 

### Box 4:

In [187]:
# The preamble modulated bits
preamble_bits_modulated = np.array([0,0,0,0,0,0,1,1,-1,\
                  -1,1,1,-1,1,-1,1,1,1,\
                  1,1,1,-1,-1,1,1,-1,1,\
                  -1,1,1,1,1,0,1,-1,-1,\
                  1,1,-1,1,-1,1,-1,-1,\
                  -1,-1,-1,1,1,-1,-1,1,\
                  -1,1,-1,1,1,1,1,0,0,0,\
                  0,0])

We will start by extracting the preamble symbols and converting them back into frequency-domain using the FFT. They will be used for channel estimation. In this example, we know the exact timing of the beginning of each symbol. However, in practice, this is unknown and an appropriate timing synchronization algorithm must be used to acquire symbol timing. Also, due to oscillator offsets between transmitter and receiver and Doppler effects, a frequency synchronization stage must be performed prior to the FFT in order to minimize the potential for Inter Carrier Interference (ICI). In this lab you do not need to do these synchronizations, you only need to implement the channel estimation.

The beginning of the first preamble OFDM symbol is sample 33 i.e. immediately after the 32 sample CP, and of the 2nd one is at 96; each one being of length 64 (in this lab, we will only use 2 preamble symbols).

In the following code box, you have to extract the two received preamble OFDM symbols and save them to use them later for channel estimation.

### Box 5:

In [188]:
# YOUR CODE STARTS HERE #

# YOUR CODE ENDS HERE #

Now, let's take the FFT of each preamble in order to recover the transmitted sequence. <br>
Do not forget to use the <code>np.fft.fftshift</code> function in order to shift the 0-frequency component of the FFT to the center of the spectrum.<br>
Also, pay attention to the <code>n : int</code> parameter of the Numpy FFT function; what should its value be in this lab?

In the following code box, you have to take the FFT of the two extracted preamble symbols to use them later for channel estimation (Don't forget <code>fftshift</code>).

### Box 6:

In [189]:
# YOUR CODE STARTS HERE #

# YOUR CODE ENDS HERE #

As mentioned previously, there will be 52 data sub-carriers in our setup. Nonetheless, and as know, not all data subcarriers are usable. In our case, we will choose to eliminate the DC subcarrier bin, and keep a number of 6 guard subcarriers for the left side of the spectrum and 5 guard subcarriers for the right side of the spectrum, allowing us to minimize interference with adjacent channels. <br>

In the following code box, define the indexes that are used for data. You can later use these indexes to access only the data. 

<i>Hint: use <code>np.arange</code></i>

### Box 7:

In [190]:
# YOUR CODE STARTS HERE #

# YOUR CODE ENDS HERE #

Now let's extract the data payload OFDM symbols and perform FFTs to recover the underlying data symbols. The CP is removed because it does not contain any information.

In the following code box, you have to take FFT of all the OFDM symbols. Note that you have to omit the preamble bits because they are already taken care of. Moreover, please note that you have to remove the CP bits before taking FFT because they contain no information. Don't forget that you have to remove these CP bits for EVERY OFDM symbol. Finally save the bits after FFT in a new array to use them later. 

Hint: Use the indexed that you have defined in Box 7 to remove the guardband and the DC bin. 

### Box 8:

In [191]:
# YOUR CODE STARTS HERE #

# YOUR CODE ENDS HERE #

## Channel Estimation <a class="anchor" id="chanest"></a>

In this section, we will perform channel estimation. Recall that the channel effect is reduced to a single complex multiplication per sub-carrier. The received symbol on sub-carrier $k$, $Y[k]$, in any given OFDM symbol is given by:    

$$ Y[k] = H[k]X[k] + N[k],$$

where $X[k]$ is the $k^{th}$ transmitted data symbol, $H[k]$ is the frequency response at sub-carrier $k$ and $N[k]$ is the noise at sub-carrier $k$.  

The data symbols transmitted in the preamble symbols are known to the receiver. Therefore, we can estimate the channel by dividing through by $X[k]$, leading to:

$$ \hat{H}[k] = \frac{Y[k]}{X[k]} = H[k] + \frac{N[k]}{X[k]} = H[k] + \alpha_{k}, $$

where $\hat{H}[k]$ is the $k^{th}$ channel estimate. The quantity $W[k]/X[k]$ is an unwanted noise term which we denote as $\alpha_{k}$. Since we have two OFDM symbols in the preamble, we can generate two channel estimates. You have to extract the symbols on the data sub-carriers, since the null sub-carriers don't carry any information.

In the following code box, you have to extract the data subcarriers from the two preamble symbols. 

Hint: Note that you already took the FFT of the preamble bits in Box 6. You should use the indexes calculated in Box 7 to get rid of the guardbands and DC bin. 

### Box 9:

In [192]:
# YOUR CODE STARTS HERE #

# YOUR CODE ENDS HERE #

In the following box, you have to calculate the channel estimation using the received preamble bits and the actual preamble bits. You only have to divide the result of Box 9 and the modulated preamble bits defined in Box 4. Don't forget to also apply the indexes calculated in Box 7 to the modulated preamble bits as well. Do this for both preambles.

### Box 10:

In [193]:
# YOUR CODE STARTS HERE #

# YOUR CODE ENDS HERE #

Since the channel effect is not changing over time (in this simplified case), we can average $h_{1}$ and $h_{2}$ to produce a final channel estimate.

In the following box, you have to average the two estimated channels calculated in Box 10.

### Box 11:

In [194]:
# YOUR CODE STARTS HERE #

# YOUR CODE ENDS HERE #

## Channel Equalisation <a class="anchor" id="chaneq"></a>

In OFDM, there are two common equalization methods; the one-tap or Zero Forcing (ZF) equalizer and the Minimum Mean Square Error (MMSE) equalizer. In this section, we will restrict our discussion to the ZF equalizer.

The one-tap or ZF equalizer is the most computationally efficient equalization method. It is given by: 

$$ \hat{X}[k] = \frac{Y[k]}{\hat{H}[k]}. $$

Before expanding the above equation, let's assume that sufficient time averaging of channel estimates has been performed such that $\alpha_{k} = 0$. This will simplify the expanded expression. Therefore, we arrive at:

$$ \hat{X}[k] = \frac{Y[k]}{\hat{H}[k]} = \frac{X[k]H[k] + N[k]}{H[k]} = X[k] + \frac{N[k]}{H[k]}.$$

Inspecting the above, we see that the equalized symbol includes an error term, $N[k]/H[k]$. This error term has a detrimental effect on the performance of the ZF equalizer because when $|H[k]|$ is close to zero, i.e. when the channel is in a deep fade, the $N[k]$ or noise term is amplified. Moreover, as the noise power increases, the amplification has a more severe effect. These issues lead to an SNR degradation after the ZF equalizer.

Based on the procedure for complex division, the ZF equalizer can be re-written as:

$$ \hat{X}[k] = Y[k]\frac{\hat{H}^{*}[k]}{|\hat{H}[k]|^{2}}, $$

where * denotes complex conjugation. The term $\frac{\hat{H}^{*}[k]}{|\hat{H}[k]|^{2}}$ is referred to as the ZF equaliser gain. 

Let's now inspect the received constellation before applying the ZF equalizer.  

In the following box, you have to plot the extracted data (in Box 8), to see that the data is totally distorted before doing the channel equalization. 

Hint: Use the scatterplot to plot the real and imaginary part of the signal in 2D. 

### Box 12:

In [195]:
# YOUR CODE STARTS HERE #

# YOUR CODE ENDS HERE #

As you can see in the plot, it is clear that the received constellations are heavily distorted. Without equalization, this would lead to a large number of symbol and bit errors. In the next box, we will equalize the data symbols using the channel estimate from the previous section.

In the next box, you have to implement the channel equalization for ALL OFDM symbols using the estimated channel in Box 11 and the explanation in the Channel Equalisation section. 

### Box 13:

In [196]:
# YOUR CODE STARTS HERE #

# YOUR CODE ENSDS HERE #

In the following box, you have to plot the data after doing the channel equalization. Do not forget to make the x-axis and y-axis using the same square scale.  

### Box 14:

In [197]:
# YOUR CODE STARTS HERE #

# YOUR CODE ENDS HERE #

Now, it's time to demodulate the data symbols we obtained using BPSK. First, implement a simple hard-decision demapper for BPSK symbols. The previous constellation plot might be useful in determining a good threshold. Then demodulate based on this threshold. Note that in our demodulation scheme, the 0 bits are mapped to -1, and the 1 bits are mapped to +1. Please append all the demodulated data (0 and 1 bits) into an array. This array will be used in the final part of the lab. 

### Box 15:

In [198]:
# YOUR CODE STARTS HERE #

# YOUR CODE ENDS HERE #

We then provide you with a small piece of code which takes a Numpy array of floating-point 1 or 0s and converts it to their respective UTF-8 characters.

In the following box, you only have to give your array calculated in Box 15 to the defined function. Then see if you have done everything right or not. 

### Box 16:

In [199]:
# Function to convert bits to text 
def bits_to_text(text_bits):
    text_bits = text_bits.astype(int)
    byte_array = np.packbits(text_bits)
    utf8_string = None
    try:
        utf8_string = byte_array.tobytes().decode('utf-8')
    except:
        print("The demodulation process didn't go as expected, please double-check your code.")
    print(utf8_string)
    
# YOUR CODE STARTS HERE #

# YOUR CODE ENDS HERE #

If you see the <code>"The demodulation process didn't go as expected, please double-check your code."</code> message, it means that you did something wrong. Please double-check and fix the problems. 